# Download data
This code finds data on yahoo finance and downloads it into appropriate folders to run the other code. To prevent an excessive number of calls on the database or reduce runtime, it may be desirable to only add part of the possible options to the "dir_and_files" dictionary.

In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf

In [19]:
# If we want to get replicable results, we should reuse the same end date, 
# though currently we want to use the latest available data
end_date = pd.datetime(year=2024, month=6, day=21)
# Only want companies that start before this date
start_date_needed = pd.datetime(year=2011, month=1, day=1)

C:\Users\rlamboll\AppData\Local\Temp\ipykernel_5324\979018772.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  end_date = pd.datetime(year=2024, month=6, day=21)
C:\Users\rlamboll\AppData\Local\Temp\ipykernel_5324\979018772.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  start_date_needed = pd.datetime(year=2011, month=1, day=1)


In [3]:
dirloc = "./input/Assorted/"
for company in ["^SPX"]:
    companydata = yf.Ticker(company)
    # get historical market data
    hist = companydata.history(start="1990-01-01", end=end_date)
    if not os.path.exists(dirloc):
        os.makedirs(dirloc)
    hist.index = hist.index.tz_localize(None).strftime('%Y-%m-%d')
    hist.to_csv(dirloc + company + ".csv")

In [16]:
dir_and_files = {}
### Or run the following to enable more robustness checks: 
"""
dir_and_files = {}{
    "renewable_20": [
    "0916.HK", "BEP", "EDPR.LS", "FSLR", "NEE", "NHPC.NS", "SUZLON.NS", "VWS.CO", "NPI.TO", "009830.KS",
    "ORA", "3800.HK", "PLUG", "NDX1.F", "BLX.TO", "ECV.F", "SLR.MC", "S92.F", "VBK.F", "CSIQ", 
    ],
    "constsust_20": [
        "ABT", "AMZN", "AZN", "BAC", "BRK-B", "COST", "GOOG", "JNJ", "JPM", "KO", 
        "LLY", "MCD", "MRK", "NESN.SW", "NVO", "PEP", "PG", "ROG.SW", "TYT.L", "WMT"
    ],
    "greencred_20": [
        "005930.KS", "AAPL", "ACN", "ADBE", "AMD", "ASML", "AVGO", "CRM", "HD", "MA", 
        "MC.PA", "MSFT", "NFLX", "NVDA", "NVS", "ORCL", "RMS.PA", "TMO", "UNH", "V"
    ], 
    "fossil_20": [
        "XOM", "CVX", "SHEL", "601857.SS", "TTE", "COP", "BP", "PBR", "EQNR",  "600028.SS",
        "0883.HK", "SO", "ENB", "SLB", "DUK", "EOG", "CNQ", "EPD", "E", "OXY"
    ],
    "control_20": [
        "AAPL", "AMZN", "BRK-B", "GOOG", "LLY", "MSFT", "NVDA", "TSM", "UNH", "V",
        "HD", "PG", "005930.KS", "MC.PA", "JNJ", "MA", "WMT", "AVGO", "NVO", "JPM"
    ], 
    "dirty_20": [
        'XOM', '600519.SS', 'CVX', 'KO', 'RELIANCE.NS',
        'SHEL', '601857.SS', 'WFC', '601288.SS', '601988.SS', 'BA', 'COP',
        'RIO', 'PBR', 'BP', '601088.SS', 'EQNR', 'MO', 'CNQ', 'ITC.NS'
    ]
}
for dir, companies in dir_and_files.items():
    print(dir)
"""

'\ndir_and_files = {}{\n    "renewable_20": [\n    "0916.HK", "BEP", "EDPR.LS", "FSLR", "NEE", "NHPC.NS", "SUZLON.NS", "VWS.CO", "NPI.TO", "009830.KS",\n    "ORA", "3800.HK", "PLUG", "NDX1.F", "BLX.TO", "ECV.F", "SLR.MC", "S92.F", "VBK.F", "CSIQ", \n    ],\n    "constsust_20": [\n        "ABT", "AMZN", "AZN", "BAC", "BRK-B", "COST", "GOOG", "JNJ", "JPM", "KO", \n        "LLY", "MCD", "MRK", "NESN.SW", "NVO", "PEP", "PG", "ROG.SW", "TYT.L", "WMT"\n    ],\n    "greencred_20": [\n        "005930.KS", "AAPL", "ACN", "ADBE", "AMD", "ASML", "AVGO", "CRM", "HD", "MA", \n        "MC.PA", "MSFT", "NFLX", "NVDA", "NVS", "ORCL", "RMS.PA", "TMO", "UNH", "V"\n    ], \n    "fossil_20": [\n        "XOM", "CVX", "SHEL", "601857.SS", "TTE", "COP", "BP", "PBR", "EQNR",  "600028.SS",\n        "0883.HK", "SO", "ENB", "SLB", "DUK", "EOG", "CNQ", "EPD", "E", "OXY"\n    ],\n    "control_20": [\n        "AAPL", "AMZN", "BRK-B", "GOOG", "LLY", "MSFT", "NVDA", "TSM", "UNH", "V",\n        "HD", "PG", "005930.KS"

In [18]:
top_companies = pd.read_csv("./input/companiesmarketcap.com - Companies ranked by Market Cap - CompaniesMarketCap.com.csv")
top_companies

,Rank,Name,Symbol,marketcap,price (USD),country,Exclude,Sustainalytics value,Sustainalytics 20-30,Sustainalytics <20,Sustainalytics >30,Unnamed: 11,34
0,1,Apple,AAPL,2.870000e+12,184.225,United States,NaN,16.7,0.0,1.0,0.0,NaN,NaN
1,2,Microsoft,MSFT,2.830000e+12,381.240,United States,NaN,15.4,0.0,1.0,0.0,NaN,NaN
2,3,Saudi Aramco,2222.SR,2.120000e+12,8.780,Saudi Arabia,FF and too new,NaN,0.0,0.0,0.0,NaN,NaN
3,4,Alphabet (Google),GOOG,1.780000e+12,143.250,United States,NaN,24.2,1.0,0.0,0.0,NaN,NaN
4,5,Amazon,AMZN,1.590000e+12,153.560,United States,NaN,29.9,1.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7968,7969,Ozon,OZON.ME,0.000000e+00,32.020,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7969,7970,Emirates NBD Bank PJSC,EMIRATESNBD.AE,0.000000e+00,0.000,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7970,7971,Emaar Properties,EMAAR.AE,0.000000e+00,0.000,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7971,7972,Energy Harbor,ENGH,0.000000e+00,0.000,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Filter out scenarios which are invalid for some reason (usually bad start dates or not listed on normal stock exchanges)
top_companies = top_companies[top_companies["Exclude"]!=top_companies["Exclude"]]
# One of these companies will fail upon load later, so we use 101 to get top 100
top_companies = top_companies[:101]
top_companies

,Rank,Name,Symbol,marketcap,price (USD),country,Exclude,Sustainalytics value,Sustainalytics 20-30,Sustainalytics <20,Sustainalytics >30,Unnamed: 11,23
0,1,Apple,AAPL,2.870000e+12,184.2250,United States,NaN,16.7,0.0,1.0,0.0,NaN,NaN
1,2,Microsoft,MSFT,2.830000e+12,381.2400,United States,NaN,15.4,0.0,1.0,0.0,NaN,NaN
3,4,Alphabet (Google),GOOG,1.780000e+12,143.2500,United States,NaN,24.2,1.0,0.0,0.0,NaN,NaN
4,5,Amazon,AMZN,1.590000e+12,153.5600,United States,NaN,29.9,1.0,0.0,0.0,NaN,NaN
5,6,NVIDIA,NVDA,1.340000e+12,541.3500,United States,NaN,13.5,0.0,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,110,Goldman Sachs,GS,1.240000e+11,380.2500,United States,NaN,24.2,0.0,1.0,0.0,NaN,NaN
110,111,Prologis,PLD,1.240000e+11,134.1750,United States,NaN,10.6,0.0,1.0,0.0,NaN,NaN
111,112,Raytheon Technologies,RTX,1.240000e+11,85.9172,United States,NaN,29.2,0.0,1.0,0.0,NaN,NaN
112,113,Booking Holdings (Booking.com),BKNG,1.230000e+11,3525.9800,United States,NaN,17.2,0.0,1.0,0.0,NaN,NaN


In [14]:
dir_and_files["all_companies"] = top_companies["Symbol"].values

In [12]:
# Two of these files won't be available from the startoo
dir_and_files["constsust_34"] = top_companies.loc[top_companies["Sustainalytics 20-30"]==1, "Symbol"][:36].values

In [20]:
# Two of these files won't be available from the startoo
dir_and_files["greencred_34"] = top_companies.loc[top_companies["Sustainalytics <20"]==1, "Symbol"][:34].values

In [43]:
top_ffs = pd.read_csv("./input/companiesmarketcap.com - Largest oil and gas companies by market cap.csv")
top_ffs = top_ffs[top_ffs["Exclude"]!=top_ffs["Exclude"]]
# Several of these companies start too late, so we use more to get top 100
top_ffs = top_ffs[:123]
dir_and_files["fossil_100"] = top_ffs["Symbol"].values

In [75]:
top_rens = pd.read_csv("./input/companiesmarketcap.com - Largest renewable energy companies by market cap.csv")
top_rens = top_rens[top_rens["Exclude"]!=top_rens["Exclude"]]
dir_and_files["renewable_all"] = top_rens["Symbol"].values

In [79]:
top_rens

,Rank,Name,Symbol,marketcap,price (USD),country,Exclude,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,Nextera Energy,NEE,1.260000e+11,61.5000,United States,NaN,1.0,NaN,NaN,NaN
2,3,Vestas Wind Systems,VWS.CO,2.859437e+10,28.3300,Denmark,NaN,2.0,NaN,NaN,NaN
4,5,EDP Renováveis,EDPR.LS,2.038676e+10,19.8400,Spain,NaN,3.0,NaN,NaN,NaN
5,6,First Solar,FSLR,1.802138e+10,168.6700,United States,NaN,4.0,NaN,NaN,NaN
6,7,Brookfield Renewable Partners,BEP,1.763838e+10,26.8100,Bermuda,NaN,5.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
84,85,Heliogen,HLGN,1.223088e+07,2.0700,United States,NaN,NaN,NaN,NaN,NaN
85,86,Capstone Green Energy,CGRN,6.788686e+06,0.3700,United States,NaN,NaN,NaN,NaN,NaN
86,87,VivoPower,VVPR,4.824346e+06,1.6700,United Kingdom,NaN,NaN,NaN,NaN,NaN
87,88,Ascent Solar Technologies,ASTI,2.979517e+06,0.8746,United States,NaN,NaN,NaN,NaN,NaN


In [21]:
for dir, companies in dir_and_files.items():
    directory = "./input/" + dir + "/"
    os.makedirs(directory, exist_ok=True)
    for company in companies:
        try:
            companydata = yf.Ticker(company)
            # get historical market data
            hist = companydata.history(start="1990-01-01", end=end_date)
            if hist.index[0].tz_localize(None) < start_date_needed:
                hist.index = hist.index.tz_localize(None).strftime('%Y-%m-%d')
                if any(hist.Close < 0):
                    negend = np.max(hist[hist.Close < 0].index)
                    print(f"Negative values in company {company} before {np.max(hist[hist.Close < 0].index)}")
                    hist = hist[hist.index > negend]
                hist.to_csv(directory + company + ".csv")
            else:
                print(f"Company {company} data not available until {hist.index[0]}")
        except:
            print(f"Failed to get {company} data")